# Variables

In [167]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [168]:
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
import pandas as pd

# Testing

In [169]:
df_task = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [ ]:
task = df_task["Task"][1]

In [171]:
task

"Bối cảnh của vấn đề:\nĐiều quan trọng là các công ty thẻ tín dụng có thể nhận ra các giao dịch thẻ tín dụng gian lận để khách hàng không phải trả tiền cho những mặt hàng mà họ không mua. Một vấn đề của task này là dữ liệu training khá là ít cho giao dịch gian lận gây vấn đề mất cân bằng nhãn dữ liệu.\n\nYêu cầu cụ thể cần đạt được:\nXây dựng một hệ thống có khả năng nhận đầu vào là một list các features liên quan đến sự gian lận của giao dịch và đầu ra hệ thống là phân loại giao dịch cụ thể đó có gian lận hay không\nMô hình cần xử lý đầu vào là 1 list các feature liên quan đến gian lận của giao dịch\nĐầu ra là nhãn của giao dịch đó thuộc về\n\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\nMột file test.csv. Mỗi hàng là features cho từng task bao gồm các cột:\n28 đặc trưng V1, V2, … V28 là các thành phần chính thu được từ PCA (Không được công bố cụ thể);  hai đặc trưng không được biến đổi bằng PCA là 'Time' và 'Amount'. Đặc trưng 'Time' thể hiện số giây đã trôi qua giữa mỗi giao dị

In [172]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [173]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    scrape_status = scraper.scrape_url(
        url,
        formats=['markdown']
    )
    return scrape_status.markdown

In [174]:
class WebScraperParameters(BaseModel):
    url: str

In [175]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [176]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [177]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL
)

In [178]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )

In [179]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [180]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
                    You are an expert in task formulation. Given a machine learning task, you will
                    help divide the task into these subtasks:
                    - Subtask1: for classifying the problem type and task description
                    - Subtask2: for identifying the input of the problem
                    - Subtask3: for identifying the output of the problem
                    describe the each subtask as specifically as possible
                    """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [181]:
state = {
    "task": HumanMessage(content=task),
}

In [182]:
state = classify_agent(state)

In [ ]:
def modeling_agent(state: AgentState) -> AgentState:
    system_prompt = f"""
                    You are a machine learning expert assigned to select the **best-fit model** for a given task.

                    ---

                    ## 🎯 Your Goal

                    Given:
                    - A user task description
                    - A list of available models (names and links)
                    - A tool that fetches model details from a provided URL (description, input/output format, usage code)

                    Your job is to:
                    1. Analyze the task and identify what type of problem it is (e.g., classification, regression, NLP, etc.)
                    2. Select a small number of candidate models whose **name or purpose suggests relevance**
                    3. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🧪 Code sample (usage code snippet)

                    ---

                    ## ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ---

                    ## 📦 Available Models:

                    {format_model_list(df_model)}

                    ---

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>
                    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    result = executor.invoke({"input": task_input})

    return {"output": result["output"]}

In [184]:
res = modeling_agent(state)

/tmp/ipykernel_34131/7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
/tmp/ipykernel_34131/7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])




> Entering new AgentExecutor chain...



Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/zhaospei/Model_4'}`
responded: To address the task of building a system that can identify fraudulent credit card transactions, we first need to understand the nature of the problem. It's a classification task where the goal is to predict whether a transaction is fraudulent or not based on a list of features related to the transaction.

Given the problem description, the task seems to involve handling imbalanced data, where instances of fraudulent transactions are less common than legitimate ones. This is a common challenge in fraud detection.

The input to the model is a list of features related to the transaction, specifically:
- 28 features (V1, V2, …, V28) derived from PCA
- 'Time': the number of seconds elapsed between the transaction and the first transaction in the dataset
- 'Amount': the amount of the transaction

The output is a classification of the transaction as either fraudulent (1) or not (0).

Let's analyze t

In [185]:
import pprint
pprint.pprint(res)

{'output': '**✅ Model name and link**  \n'
           ' `Phân loại gian lận thẻ tín dụng` — '
           '`https://huggingface.co/zhaospei/Model_4`\n'
           '\n'
           ' **📝 Description (from tool):**  \n'
           ' Đây là mô hình nhị phân được huấn luyện để phát hiện gian lận '
           'trong giao dịch thẻ tín dụng, sử dụng tập dữ liệu từ Kaggle Credit '
           'Card Fraud Detection. Mô hình được xây dựng bằng Keras và xử lý '
           'vấn đề dữ liệu mất cân bằng nghiêm trọng (chỉ 0.18% giao dịch là '
           'gian lận).\n'
           '\n'
           ' **📥 Input format (from tool):**  \n'
           ' Shape: [batch_size, 30]  \n'
           ' Kiểu dữ liệu: np.ndarray hoặc tf.Tensor  \n'
           ' Chuẩn hóa: Dữ liệu đầu vào cần được chuẩn hóa tương tự như trong '
           'quá trình huấn luyện\n'
           '\n'
           ' **📤 Output format (from tool):**  \n'
           ' Shape: [batch_size, 1]  \n'
           ' Ý nghĩa: Xác suất giao dịch là gian lận 

In [186]:
print(res["output"])


**✅ Model name and link**  
 `Phân loại gian lận thẻ tín dụng` — `https://huggingface.co/zhaospei/Model_4`

 **📝 Description (from tool):**  
 Đây là mô hình nhị phân được huấn luyện để phát hiện gian lận trong giao dịch thẻ tín dụng, sử dụng tập dữ liệu từ Kaggle Credit Card Fraud Detection. Mô hình được xây dựng bằng Keras và xử lý vấn đề dữ liệu mất cân bằng nghiêm trọng (chỉ 0.18% giao dịch là gian lận).

 **📥 Input format (from tool):**  
 Shape: [batch_size, 30]  
 Kiểu dữ liệu: np.ndarray hoặc tf.Tensor  
 Chuẩn hóa: Dữ liệu đầu vào cần được chuẩn hóa tương tự như trong quá trình huấn luyện

 **📤 Output format (from tool):**  
 Shape: [batch_size, 1]  
 Ý nghĩa: Xác suất giao dịch là gian lận (giá trị từ 0 đến 1)

 **🧪 Example code (from tool):**  
 ```python
import numpy as np
import tensorflow as tf
from huggingface_hub import from_pretrained_keras

# Tải mô hình từ Hugging Face Hub
model = from_pretrained_keras("zhaospei/Model_4")
model.summary()

# Tạo dữ liệu đầu vào mẫu (g